In [72]:
import torch
import numpy as np
import torch.nn.functional as F
a= torch.rand(4,4, 3)
torch.sigmoid(a)


tensor([[[0.5575, 0.5457, 0.6759],
         [0.6426, 0.5401, 0.7224],
         [0.5039, 0.6169, 0.5694],
         [0.5044, 0.6592, 0.5000]],

        [[0.6185, 0.6470, 0.7081],
         [0.5825, 0.6417, 0.6335],
         [0.6799, 0.5593, 0.7011],
         [0.6615, 0.6089, 0.6963]],

        [[0.6433, 0.5451, 0.6221],
         [0.5189, 0.6083, 0.6928],
         [0.5525, 0.5980, 0.6153],
         [0.6191, 0.5119, 0.6352]],

        [[0.6917, 0.5375, 0.5412],
         [0.6235, 0.5627, 0.6107],
         [0.7020, 0.5184, 0.5431],
         [0.5916, 0.6807, 0.7174]]])

In [73]:
def relu(x):
    x_ = x.copy()
    x_[x_<0] = 0
    return x_

class AdjustContraste():
    def __init__(self):
        self.num_parameters = 1
        self.window_names = ["parameter"]
        self.slider_names = ["contrast"]

    def __call__(self, list_editted, parameters):     
        editted  =  list_editted.numpy()

        mean = editted.mean()
        editted_ = (editted-mean)*(parameters[0]+1)+mean
        editted_ = relu(editted_)
        editted_ = 1-relu(1-editted_)
        return [editted_]

In [79]:
old = AdjustContraste()(a,[0.5])[0]

In [25]:
sigmoid_inverse(a)

/tmp/ipykernel_103433/3352330405.py:12: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  return -np.log(1/(y_)-1)


tensor([-0.8648,  0.8165, -1.3251, -0.7355])

In [40]:
class AdjustContrast():
    def __init__(self):
        self.num_parameters = 1
        self.window_names = ["parameter"]
        self.slider_names = ["contrast"]

    def __call__(self, images:torch.Tensor, parameters:torch.Tensor):
        mean = images.mean()
        editted = (images-mean)*(parameters[0]+1)+mean
        editted = F.relu(editted)
        editted= 1-F.relu(1-editted)
        return [editted]

In [38]:
torch.Tensor([0.5]).dtype

torch.float32

In [76]:
new = AdjustContrast()(a,torch.Tensor([0.5]))

In [80]:
old==new

array([[[[ True,  True,  True],
         [ True,  True,  True],
         [ True,  True,  True],
         [ True,  True,  True]],

        [[ True,  True,  True],
         [ True,  True,  True],
         [ True,  True,  True],
         [ True,  True,  True]],

        [[ True,  True,  True],
         [ True,  True,  True],
         [ True,  True,  True],
         [ True,  True,  True]],

        [[ True,  True,  True],
         [ True,  True,  True],
         [ True,  True,  True],
         [ True,  True,  True]]]])